라이브러리 설치

In [1]:
# Streamlit, LangChain 등 앱 실행에 필요한 라이브러리 설치
!pip install streamlit pyngrok langchain-core langchain langchain_openai langchain_community tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00


기본 설정 (API 키 및 드라이브 연결)

In [2]:
#  기본 설정 (드라이브 마운트 및 API 키)
import os
from google.colab import drive
from google.colab import userdata

# 구글 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

# 1. Colab 보안 비밀에서 API 키를 가져옵니다.
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ Colab 보안 비밀에서 API 키를 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"❌ Colab 보안 비밀에서 API 키를 불러오는 데 실패했습니다: {e}")
    # 키가 없으면 아래 코드가 의미 없으므로 중단합니다.
    raise SystemExit("API 키를 먼저 설정해주세요.")

# 2. Streamlit이 읽을 수 있도록 .streamlit 폴더와 secrets.toml 파일을 생성합니다.
# 이 작업은 Colab 노트북 환경에서 수행됩니다.
!mkdir -p .streamlit
with open(".streamlit/secrets.toml", "w") as f:
    f.write(f'OPENAI_API_KEY = "{OPENAI_API_KEY}"')

print("✅ Streamlit용 비밀 파일(secrets.toml) 생성이 완료되었습니다.")

Mounted at /content/drive
✅ Colab 보안 비밀에서 API 키를 성공적으로 불러왔습니다.
✅ Streamlit용 비밀 파일(secrets.toml) 생성이 완료되었습니다.


 Streamlit 앱 코드 작성 (app.py 파일 생성)

In [3]:
%%writefile app.py

import streamlit as st
import os
import datetime
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# --- 1. API 키 설정 ---
try:
    os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]
except KeyError:
    st.error("OpenAI API 키가 설정되지 않았습니다. .streamlit/secrets.toml 파일이 올바르게 생성되었는지 확인해주세요.")
    st.stop()

# --- 2. RAG 체인 로딩 함수 (캐시 사용) ---
@st.cache_resource
def load_rag_chain():
    VECTOR_STORE_DIR = '/content/drive/MyDrive/RAG_Audit_Project/03_vector_store'
    embeddings_model = OpenAIEmbeddings()
    vectorstore = FAISS.load_local(
        VECTOR_STORE_DIR,
        embeddings_model,
        allow_dangerous_deserialization=True
    )
    prompt_template = """
    당신은 30년 이상의 경력을 가진 대한민국 공인회계사(KICPA) 파트너입니다.
    당신의 임무는 주어진 감사 사례(질문)에 대해, 오직 제공된 Context(감사 기준서 및 교재) 내용을 근거로 명확하고 구조화된 답변을 제공하는 것입니다.
    단, 질문의 의도를 해석하고 무엇을 답변해야 하는지 파악할 때는 스스로 이해하려고 노력하기 바랍니다. 사례는 스스로 분석해야 합니다.
    포인트는 사례에서 무엇을 평가해야 하는지 인지하고, 기준서에 부합하지 않는 내용이 있는지 찾아내는 것입니다.
    필요한 절차를 제시할 때는 제공된 context를 근거로 그대로 대답하십시오.

    **[규칙]**
    1. 반드시 Context에 명시된 내용만을 사용해야 합니다. 당신의 사전 지식이나 외부 정보를 절대 사용하지 마세요.
    2. Context에 질문에 대한 답변 근거가 없다면, "제공된 기준서 내용만으로는 답변할 수 없습니다."라고 명확히 밝히세요.
    3. 답변은 아래의 '출력 형식'을 반드시 준수하여 한국어로 작성해주세요.
    4. 질문에서 요구하는 답변의 개수(예: 2가지)를 정확히 파악하고 그에 맞춰 답변해야 합니다.

    **[답변 형식]**
    ### 1. 결론/판단:
    - 질문에 대한 명확한 결론을 내립니다. 물어보는 것에 대해 짧게 대답합니다
    - 여부를 물었을 때는 O/X로 대답합니다.
    - 적절한가? 하고 물었을 때는 예, 아니오로 대답합니다.
    - 잘못된 감사절차를 물었을 때는 본문에서 잘못된 감사절차 내용을 찾습니다.
    - 수행하여야 할 절차를 물었을 때는 구체적인 사례보다는 context에서 절차 관련된 언급이 있을 경우 기준서를 최대한 준용합니다.
    - 몇 가지를 물어보는지 파악하고 질문에 맞는 답안을 구성합니다. (2가지를 물었을 경우 2가지로 대답합니다. 첫째 사항/절차(물음에서 물어본것)는, 둘째 사항/절차(물음에서 물어본것)는, 이런 식으로 대답합니다.)

    ### 2. 이유:
    - [Context에서 찾아낸 구체적인 기준이나 문구를 인용합니다.]
    - 최대한 감사기준서나 절차는 제공된 context의 용어와 표현을 그대로 사용합니다.
    - 사례에서 제공된 문제점을 지적할 수 있습니다.
    - 다만, context의 회계감사기준, 윤리기준 등 다양한 내용을 근거로 위배된 부분이 있는지 파악합니다.
    - 제공된 질문에서 잘못된 부분을, context를 근거로 평가해야 합니다.
    - 당신은 30년 이상의 숙련된 회계사로, 모든 회계감사기준을 명백히 이해하고 있습니다.
    - 당신에게 제시된 질문의 text에서 감사절차가 적절/부적절한지 판단한 이유를 근거를 들어 말해야 합니다.

    ---
    Context:
    {context}

    Question:
    {question}

    Answer(Korean):
    """
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(search_kwargs={'k': 5}),
        chain_type_kwargs={"prompt": PROMPT}, return_source_documents=True
    )
    return rag_chain

# --- 3. 대화 내용을 HTML로 변환하는 함수 ---
def generate_html(history):
    # (이전과 동일)
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <title>회계감사 AI 대화 기록</title>
    <style>
        body {{ font-family: sans-serif; line-height: 1.6; padding: 20px; }}
        .container {{ max-width: 800px; margin: auto; border: 1px solid #ddd; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
        .question {{ background-color: #e1f5fe; padding: 15px; border-radius: 8px; margin-bottom: 10px; border-left: 5px solid #0288d1; }}
        .answer {{ background-color: #f1f8e9; padding: 15px; border-radius: 8px; margin-bottom: 20px; border-left: 5px solid #7cb342;}}
        h1, h2, h3 {{ color: #0277bd; }}
        h1 {{ text-align: center; }}
        .timestamp {{ text-align: center; color: #757575; margin-bottom: 20px; }}
        pre {{ white-space: pre-wrap; word-wrap: break-word; font-size: 14px; }}
    </style>
    </head>
    <body>
        <div class="container">
            <h1>회계감사 AI 대화 기록</h1>
            <p class="timestamp">저장 시각: {timestamp}</p>
    """
    for item in history:
        # Markdown을 HTML에서 줄바꿈이 되도록 처리
        formatted_result = item['result'].replace('\\n', '<br>')
        html_content += f"<h2>질문:</h2><div class='question'><pre>{item['query']}</pre></div>"
        html_content += f"<h2>AI 답변:</h2><div class='answer'>{formatted_result}</div><hr>"

    html_content += "</div></body></html>"
    return html_content

# --- 4. Streamlit 웹 UI 구성 ---
st.set_page_config(page_title="회계감사 RAG AI", layout="wide")
st.title("🤖 회계감사 RAG AI 어시스턴트")

# st.session_state를 사용하여 대화 기록을 세션 간에 유지
if 'history' not in st.session_state:
    st.session_state.history = []

# --- 사이드바 UI ---
with st.sidebar:
    st.header("메뉴")

    # 대화 기록 초기화 버튼
    if st.button("대화 기록 초기화"):
        st.session_state.history = []
        st.success("대화 기록이 초기화되었습니다.")

    # 대화 기록이 있을 때만 다운로드 버튼 표시
    if st.session_state.history:
        html_str = generate_html(st.session_state.history)
        file_name = f"audit_conversation_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
        st.download_button(
            label="대화 내용 HTML로 저장",
            data=html_str.encode('utf-8'),
            file_name=file_name,
            mime='text/html'
        )

# --- 메인 화면 UI ---
try:
    rag_chain = load_rag_chain()

    query = st.text_area("공인회계사 2차시험 회계감사 문제를 입력하세요:", height=150, placeholder="여기에 감사 사례를 입력해주세요...")

    if st.button("답변 생성하기"):
        if query:
            with st.spinner('AI가 감사 기준서를 검토하며 답변을 생성 중입니다...'):
                response = rag_chain.invoke(query)
                # 대화 기록을 session_state에 저장
                st.session_state.history.append({'query': query, 'result': response['result']})
        else:
            st.warning("질문을 입력해주세요!")

    # 최신 답변을 화면 상단에 표시
    if st.session_state.history:
        st.markdown("---")
        st.subheader("최근 질문과 답변")
        latest_item = st.session_state.history[-1]
        st.info(f"**Q:** {latest_item['query']}")
        st.success(f"**A:** {latest_item['result']}")

except Exception as e:
    st.error(f"앱 실행 중 오류가 발생했습니다: {e}")

Writing app.py


Streamlit 앱 실행

In [4]:
from pyngrok import ngrok
from google.colab import userdata

# ngrok 인증 토큰 설정
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("Ngrok 인증 토큰 설정 완료.")
except Exception as e:
    print(f"Ngrok 인증 토큰 설정 실패: {e}")

# 앱 실행 및 접속 주소 생성
try:
    ngrok.kill()
    public_url = ngrok.connect(8501)
    print(f"🎉 Streamlit 앱이 준비되었습니다! 아래 주소로 접속하세요: \n{public_url}")
    !streamlit run app.py &>/dev/null&
except Exception as e:
    print(f"Streamlit 앱 실행 실패: {e}")

Ngrok 인증 토큰 설정 완료.
🎉 Streamlit 앱이 준비되었습니다! 아래 주소로 접속하세요: 
NgrokTunnel: "https://3a7f274b32dc.ngrok-free.app" -> "http://localhost:8501"
